In [1]:
import sys
sys.path.append('../src-py/')

In [2]:
from generate_conclusions import *

In [3]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

pd.set_option('display.max_colwidth', None)

In [4]:
import torch

In [5]:
torch.__version__

'1.9.0+cu111'

In [6]:
gen_kwargs = {
    "do_sample": True, 
    "max_length":200,
    "top_p":0.95, 
    "top_k":50,
}

### Generate conclusions for the comments on the training dataset:

We might probably use these auto-generated conclusions to train the model instead of taking the whole comment

In [4]:
train_df = pd.read_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/train_conclusion_comp_remove_75sem_perc.pkl')

In [5]:
len(train_df)

92397

In [6]:
train_df.head()

,post_id,split,comment_id,title,post,n_sentences,counter,num_cand_conc,masked_premises,premises_with_conclusion
0,t3_1u4mmo,train,t1_ceehlbn,I believe that churches and other religious institutions should have to pay taxes.,"[i believe that a church is like any other profit making business and therefore most pay similar taxes., i am not a religious person myself so i do not know the complete logistics of the revenue and spending of a church but i do understand they can yield a good amount of cash., not entirely sure how accurate this article is, but it states that we could gain an additional 71 billion dollars per year and in a time where we as a country are in trillions of dollars of debt taxing a church seems reasonable to me., cmv]",4,"[i have heard this before ofc but when it is said louder than, or separate from the same issue when applied to corporations, or other non profits, i just think it's just religious discrimination., most, if not all corporations pay a much smaller percentage of their income in taxes than i do., why not focus on closing corporate loopholes first?, it would generate a lot more income than targeting religious institutions., why not focus on non religious nonprofits?, tons of those are shams where the head of said organization makes 7 figures., many are politically motivated, and actually are for profit in the sense that they are for changing laws and policies that allow third parties to become more profitable., i just don't have much respect for the taxing religions thing because you can apply the same arguments to different organizations resulting in higher tax revenue., i mean, sure., if some tv preacher is pulling in 7 figures selling 'prayer rags' then yeah, close whatever loopholes make that possible., but at the same time you have to close those same loopholes for non religious nonprofits, else it's just discrimination.]",1,"[i believe that a church is like any other profit making business and therefore most pay similar taxes., i am not a religious person myself so i do not know the complete logistics of the revenue and spending of a church but i do understand they can yield a good amount of cash., not entirely sure how accurate this article is, but it states that we could gain an additional 71 billion dollars per year and in a time where we as a country are in trillions of dollars of debt taxing a church seems reasonable to me., cmv]","[i believe that a church is like any other profit making business and therefore most pay similar taxes., i am not a religious person myself so i do not know the complete logistics of the revenue and spending of a church but i do understand they can yield a good amount of cash., not entirely sure how accurate this article is, but it states that we could gain an additional 71 billion dollars per year and in a time where we as a country are in trillions of dollars of debt taxing a church seems reasonable to me., cmv, Therefore, I believe that churches and other religious institutions should have to pay taxes. ]"
1,t3_1u4mmo,train,t1_ceei77d,I believe that churches and other religious institutions should have to pay taxes.,"[i believe that a church is like any other profit making business and therefore most pay similar taxes., i am not a religious person myself so i do not know the complete logistics of the revenue and spending of a church but i do understand they can yield a good amount of cash., not entirely sure how accurate this article is, but it states that we could gain an additional 71 billion dollars per year and in a time where we as a country are in trillions of dollars of debt taxing a church seems reasonable to me., cmv]",4,"[one of the historical methods of repressing out of favor religions is to make them pay exorbitant taxes., the blanket religious tax exemption is one of the ways we comply with the first amendment., many atheists seem to forget it guarantees the free exercise of religion in addition to forbidding the establishment of a state reli

In [ ]:
conclusions = generate_conclusion(train_df.masked_premises.tolist(), gen_kwargs, batch_size=16)

  0%|          | 0/93 [00:00<?, ?ba/s]

In [ ]:
train_df['bart_conclusion'] = conclusions

In [21]:
train_df['counter_bart_conclusion'] =  generate_conclusion(train_df.counter.tolist(), gen_kwargs, batch_size=16)

  0%|          | 0/93 [00:00<?, ?ba/s]

In [23]:
train_df.head()

,post_id,split,comment_id,title,post,n_sentences,counter,num_cand_conc,masked_premises,premises_with_conclusion,bart_conclusion,counter_bart_conclusion
0,t3_1u4mmo,train,t1_ceehlbn,I believe that churches and other religious institutions should have to pay taxes.,"[i believe that a church is like any other profit making business and therefore most pay similar taxes., i am not a religious person myself so i do not know the complete logistics of the revenue and spending of a church but i do understand they can yield a good amount of cash., not entirely sure how accurate this article is, but it states that we could gain an additional 71 billion dollars per year and in a time where we as a country are in trillions of dollars of debt taxing a church seems reasonable to me., cmv]",4,"[i have heard this before ofc but when it is said louder than, or separate from the same issue when applied to corporations, or other non profits, i just think it's just religious discrimination., most, if not all corporations pay a much smaller percentage of their income in taxes than i do., why not focus on closing corporate loopholes first?, it would generate a lot more income than targeting religious institutions., why not focus on non religious nonprofits?, tons of those are shams where the head of said organization makes 7 figures., many are politically motivated, and actually are for profit in the sense that they are for changing laws and policies that allow third parties to become more profitable., i just don't have much respect for the taxing religions thing because you can apply the same arguments to different organizations resulting in higher tax revenue., i mean, sure., if some tv preacher is pulling in 7 figures selling 'prayer rags' then yeah, close whatever loopholes make that possible., but at the same time you have to close those same loopholes for non religious nonprofits, else it's just discrimination.]",1,"[i believe that a church is like any other profit making business and therefore most pay similar taxes., i am not a religious person myself so i do not know the complete logistics of the revenue and spending of a church but i do understand they can yield a good amount of cash., not entirely sure how accurate this article is, but it states that we could gain an additional 71 billion dollars per year and in a time where we as a country are in trillions of dollars of debt taxing a church seems reasonable to me., cmv]","[i believe that a church is like any other profit making business and therefore most pay similar taxes., i am not a religious person myself so i do not know the complete logistics of the revenue and spending of a church but i do understand they can yield a good amount of cash., not entirely sure how accurate this article is, but it states that we could gain an additional 71 billion dollars per year and in a time where we as a country are in trillions of dollars of debt taxing a church seems reasonable to me., cmv, Therefore, I believe that churches and other religious institutions should have to pay taxes. ]",i believe that churches should have to pay taxes.,i think corporate tax is just religious discrimination.
1,t3_1u4mmo,train,t1_ceei77d,I believe that churches and other religious institutions should have to pay taxes.,"[i believe that a church is like any other profit making business and therefore most pay similar taxes., i am not a religious person myself so i do not know the complete logistics of the revenue and spending of a church but i do understand they can yield a good amount of cash., not entirely sure how accurate this article is, but it states that we could gain an additional 71 billion dollars per year and in a time where we as a country are in trillions of dollars of debt taxing a church seems reasonable to me., cmv]",4,"[one of the historical methods of repressing out of favor religions is to make them pay exorbitant taxes., the blanket religious tax exemption is one of the ways we comply with the firs

In [25]:
train_df.to_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/train_conclusion_comp_remove_75sem_perc_with_added_conclusions.pkl')

### Generate comclusions for the validation set:

In [8]:
valid_df = pd.read_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/valid_conclusion_comp_remove_75sem_perc_with_targets.pkl')

In [9]:
post_dict = list(pd.Series(valid_df.masked_premises.values, index=valid_df.post_id).to_dict().items())

In [10]:
post_ids, posts = zip(*post_dict)

In [11]:
len(post_ids)

2531

In [12]:
pred_conclusions = generate_conclusion(posts, gen_kwargs, batch_size=8)

  0%|          | 0/3 [00:00<?, ?ba/s]

/usr/local/lib/python3.6/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [13]:
post_pred_conclusions = {x[0]: x[1] for x in zip(post_ids, pred_conclusions)}

In [14]:
valid_df['bart_conclusion'] = valid_df.post_id.apply(lambda x: post_pred_conclusions[x])

In [15]:
valid_df[['title', 'bart_conclusion', 'post']].sample(10).head()

,title,bart_conclusion,post
410484,"People who unironically use the NPC meme represent ""the NPC"" more so than the liberals they claim are NPCs.",people who unironically use the nPC meme are basically 'nPCs'.,"[yeah, i know, popular opinion, but i do want to hear the other side., i don't think i've ever seen a unique argument by someone who unironically uses the npc meme., pretty much all of their comments are: 'npc liberals' 'orange man bad' ' lt;insert code here gt;' and there's no actual unique meme made by said users., when questioned on how repetitive they are, they claim 'liberals are the real npcs. ', i get that they're 'imitating the liberal npc,' but they don't actually respond to unique thoughts., instead, they repeat the same few phrases over and over again, therefore being more like an npc than the ones they claim are npcs., cmv.]"
356767,NATO should be dissolved.,"The united states should form a new ""police union""","[nato was formed at the end of wwii and the beginning of the cold war to deter a strong soviet union from invading a europe still in ashes., it successfully accomplished that goal., russia is now a shadow of its former self, while the european union as a whole is the wealthiest region in the world., a land invasion from the east is no longer a realistic possibility., additionally, the only occasions where article 5 has been invoked has been to invade smaller countries in response to terrorist threats dragging other countries into small conflicts that they have no business being involved in., the united states should not be the world's policeman., nato should be dissolved over time ., the eu should form its own defense, which then signs a bilateral deal with the united states.]"
317720,I think most people in this world are stupid.,people are stupid. (please read before responding),"[firstly i assure anyone that replies i am very open to change., my reasoning for my thoughts are mostly unrelated to iq and competence., most of my thoughts are based on experience and what i see around me., i think people are stupid because they are unable to think for themselves, critically think or perform any sort of self analysis., when asked why they do the things they do, they do not know as most of their decisions and actions are spontaneous and aren't made with a conscious effort., people cannot make decisions on their own and their actions are based off what others do or say., their future is decided by what trends exist and how their social circle feel., opinions are formed unconsciously and without any effort., they need to be constantly occupied and entertained by physical activities or social media, when left alone with nothing except them and their thoughts, their minds are either unoccupied or filled with meaningless thoughts like 'what should i have for dinner? ', they do not self reflect., ever., when their opinions or actions are questioned or challenged in any way shape or form, they either get emotional and or avoid the topic as a whole., i am sorry to those i have not replied to, its become a little overwhelming answering the same questions over and over.]"
303772,Religion enshrines the values of a certain time period within a society. Its effects are good or bad according to whether these values are true or false.,Religion enshrines the values of a certain time period within a society.,"[i'm speaking from the perspective of atheism and moral realism., i am trying to determine what effects religious belief generally has, so that i can determine whether and how to combat it., my current hypothesis is that religion enshrines the values of a specific time period within a society., what this means is that the religion forms at a certain point in time, so it will advocate the values present in the culture at that point in time., adherents of the religion adopt and preserve those values over the subsequent decades, centuries, or millenia because they have faith in the religion., if the values are true, the

In [16]:
valid_df.to_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/valid_conclusion_comp_remove_75sem_perc_with_targets.pkl')valid_conclusion_all_sample

### Generate conclusions for the test-dateset:

In [7]:
test_df = pd.read_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/test_conclusion_comp_remove_75sem_perc_with_targets.pkl')

In [8]:
post_dict = list(pd.Series(test_df.masked_premises.values, index=test_df.post_id).to_dict().items())
post_ids, posts = zip(*post_dict)
pred_conclusions = generate_conclusion(posts, gen_kwargs, batch_size=8)
post_pred_conclusions = {x[0]: x[1] for x in zip(post_ids, pred_conclusions)}
test_df['bart_conclusion'] = test_df.post_id.apply(lambda x: post_pred_conclusions[x])
test_df.to_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/test_conclusion_comp_remove_75sem_perc_with_targets.pkl')

  0%|          | 0/3 [00:00<?, ?ba/s]

/usr/local/lib/python3.6/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [ ]:
#Now generate conclusions using ArgLexRank 

### Generate conclusions for the valid-all dataset:

In [9]:
test_df = pd.read_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/valid_conclusion_all_sample_with_targets.pkl')

In [10]:
post_dict = list(pd.Series(test_df.post.values, index=test_df.post_id).to_dict().items())
post_ids, posts = zip(*post_dict)
pred_conclusions = generate_conclusion(posts, gen_kwargs, batch_size=8)
post_pred_conclusions = {x[0]: x[1] for x in zip(post_ids, pred_conclusions)}
test_df['bart_conclusion'] = test_df.post_id.apply(lambda x: post_pred_conclusions[x])
test_df.to_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/valid_conclusion_all_sample_with_targets.pkl')

  0%|          | 0/2 [00:00<?, ?ba/s]

In [7]:
test_df = pd.read_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/test_conclusion_all_sample_with_targets.pkl')
post_dict = list(pd.Series(test_df.post.values, index=test_df.post_id).to_dict().items())
post_ids, posts = zip(*post_dict)
pred_conclusions = generate_conclusion(posts, gen_kwargs, batch_size=8)
post_pred_conclusions = {x[0]: x[1] for x in zip(post_ids, pred_conclusions)}
test_df['bart_conclusion'] = test_df.post_id.apply(lambda x: post_pred_conclusions[x])
test_df.to_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/test_conclusion_all_sample_with_targets.pkl')

  0%|          | 0/2 [00:00<?, ?ba/s]

/usr/local/lib/python3.6/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
